In [1]:
!pip install nltk pyodbc sqlalchemy

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   -------------------- ------------------- 0.8/1.5 MB 2.2 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 2.2 MB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 932.9 kB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.1 MB 987.4 kB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.1 MB 987.4 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 931.8 kB/s eta 0:00:02
   ------------------------ -----------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
# Download the VADER lexicon for sentiment analysis if not already present.
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\91945\AppData\Roaming\nltk_data...


True

In [28]:
# Function to fetch data from SQL Server database
def fetch_data_from_sql():
    # Define the connection string (update your own server and database if needed)
    conn_str = (
        "Driver={SQL Server};"
        "Server=LAPTOP-A281U3H1;"
        "Database=MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )
    
    # Establish the connection
    conn = pyodbc.connect(conn_str)
    
    # Define your query
    query = """
    SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText 
    FROM [dbo].[customer_reviews]
    """
    
    # Load data into a pandas DataFrame
    df = pd.read_sql(query, conn)
    
    # Close the connection
    conn.close()
    
    return df


In [ ]:
# Fetch customer reviews data
customer_reviews_df = fetch_data_from_sql()

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()




C:\Users\91945\AppData\Local\Temp\ipykernel_16972\869606402.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [30]:
display(customer_reviews_df)

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."
...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special."
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.


In [31]:
# Function to calculate sentiment score from review text
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']


In [32]:
# Function to categorize sentiment based on score and rating
def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score < -0.05:
        if rating <= 2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating >= 4:
            return 'Positive'
        elif rating <= 2:
            return 'Negative'
        else:
            return 'Neutral'


In [33]:
# Function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'


In [34]:
# Apply sentiment analysis and transformations
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)


In [35]:
# Display the first few rows with sentiment details
customer_reviews_df.head()


,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive,0.0 to 0.49
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.0 to 0.49
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0


In [37]:
# Save the processed DataFrame to a CSV file
customer_reviews_df.to_csv('customer_reviews_with_sentiment.csv', index=False)
